# Strands Graph Architecture Demo

This notebook demonstrates:
- Building multi-agent workflows with GraphBuilder
- Different graph topologies (sequential, star, hierarchical)
- Agent coordination and data flow
- Observability with AgentResult and GraphResult

In [2]:
from strands import Agent, tool
from strands.multiagent.graph import GraphBuilder
from strands.models import BedrockModel

## Pattern 1: Sequential Pipeline

Plan → Execute → Synthesize pattern for structured reasoning

In [4]:
# Define tools
@tool
def search_flights(origin: str, destination: str) -> str:
    """Search available flights"""
    return f"Found 3 flights from {origin} to {destination}: $299, $349, $399"

@tool
def check_weather(city: str) -> str:
    """Check weather forecast"""
    return f"Weather in {city}: Sunny, 75°F"

# Create specialized agents
planner = Agent(
    name="planner",
    system_prompt="Create a step-by-step plan. Output format: Plan 1: <action>, Plan 2: <action>",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

executor = Agent(
    name="executor",
    system_prompt="Execute the plan using available tools. Return results for each step.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[search_flights, check_weather]
)

synthesizer = Agent(
    name="synthesizer",
    system_prompt="Synthesize execution results into a final recommendation.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

In [5]:
# Build sequential graph
builder = GraphBuilder()
builder.add_node(planner, "planner")
builder.add_node(executor, "executor")
builder.add_node(synthesizer, "synthesizer")

builder.add_edge("planner", "executor")
builder.add_edge("executor", "synthesizer")

builder.set_entry_point("planner")

sequential_graph = builder.build()

Graph without execution limits may run indefinitely if cycles exist


In [15]:
# Execute graph
result = sequential_graph("Find flights from NYC to LAX and check weather")
print("/n/n")
print("="*120)
print("=== Final Response ===")
print(result.results['synthesizer'].result)
print("="*120)
print("\n=== Execution Path ===")
for node in result.execution_order:
    print(f"{node.node_id}: {node.result.result}...")

Plan 1: Find flights from NYC to LAX
1. Open a web browser.
2. Go to a reputable flight search website (e.g., Google Flights, Expedia, Kayak, Skyscanner).
3. Enter "NYC" in the departure city field.
4. Enter "LAX" in the destination city field.
5. Select your desired travel dates (roundtrip or one-way).
6. Click on the search button to view the available flight options.
7. Review the search results, considering factors such as departure and arrival times, duration, price, and airline preferences.
8. Choose the flight that best suits your needs and click on it for more details.
9. If desired, proceed to the booking page to reserve your flight.

Plan 2: Check weather
1. Open a web browser.
2. Go to a reliable weather website (e.g., Weather.com, AccuWeather, The Weather Channel).
3. Enter "Los Angeles, CA" in the search bar.
4. Click on the search button.
5. Review the current weather conditions, including temperature, humidity, wind speed, and any weather warnings or alerts.
6. Check the

## Pattern 2: Star Topology

Central coordinator delegating to specialist agents

In [18]:
# Specialist agents
flight_specialist = Agent(
    name="flight_specialist",
    system_prompt="You specialize in flight searches and bookings.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[search_flights]
)

weather_specialist = Agent(
    name="weather_specialist",
    system_prompt="You specialize in weather forecasts.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[check_weather]
)

coordinator = Agent(
    name="coordinator",
    system_prompt="Coordinate specialist agents and synthesize their outputs.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

# Build star topology
star_builder = GraphBuilder()
star_builder.add_node(coordinator, "coordinator")
star_builder.add_node(flight_specialist, "flights")
star_builder.add_node(weather_specialist, "weather")

star_builder.add_edge("coordinator", "flights")
star_builder.add_edge("coordinator", "weather")
star_builder.add_edge("flights", "coordinator")
star_builder.add_edge("weather", "coordinator")

star_builder.set_entry_point("coordinator")

star_builder.set_max_node_executions(10)

star_graph = star_builder.build()

In [21]:
# Execute star graph
result = star_graph("Plan a trip from SFO to MIA")

print("/n/n")
print("="*120)
print("=== Final Answer ===")
print(result.results['coordinator'].result)

To plan a comprehensive trip from San Francisco International Airport (SFO) to Miami International Airport (MIA), we need to cover several key aspects: booking flights, arranging accommodations, and planning ground transportation. Here’s a detailed guide to help you through each step:

### Step 1: Booking Flights

1. **Choose Your Flight**:
   - Select your preferred flight option for both the outbound and return trips. Specify any specific preferences such as departure or arrival times, if necessary.
   - Indicate any preferred airlines or additional services you’d like to book (e.g., extra legroom, meal preferences).

2. **Flight Details**:
   - Outbound Flight (SFO to MIA): Choose a flight that fits your schedule.
   - Return Flight (MIA to SFO): Select a flight that allows ample time for your return journey.

### Step 2: Arranging Accommodations in Miami

1. **Select Your Accommodation**:
   - Choose from the following options or let me know if you have a specific hotel in mind:
  

## Pattern 3: Hierarchical DAG

Multi-level workflow with parallel branches

In [24]:
@tool
def calculate_budget(amount: float, days: int) -> str:
    """Calculate daily budget"""
    return f"Daily budget: ${amount/days:.2f}"

# Multi-level agents
research_agent = Agent(
    name="researcher",
    system_prompt="Research travel options.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[search_flights, check_weather]
)

budget_agent = Agent(
    name="budget",
    system_prompt="Analyze budget and costs.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0"),
    tools=[calculate_budget]
)

report_agent = Agent(
    name="reporter",
    system_prompt="Create final travel report from research and budget analysis.",
    model=BedrockModel(model_id="us.amazon.nova-micro-v1:0")
)

# Build hierarchical DAG
dag_builder = GraphBuilder()
dag_builder.add_node(research_agent, "research")
dag_builder.add_node(budget_agent, "budget")
dag_builder.add_node(report_agent, "report")

# Parallel branches converging to report
dag_builder.add_edge("research", "report")
dag_builder.add_edge("budget", "report")

dag_builder.set_entry_point("research")

dag_builder.set_max_node_executions(10)

dag_graph = dag_builder.build()

In [25]:
# Execute DAG
result = dag_graph("Plan 5-day trip from BOS to SEA with $2000 budget")
print("/n/n")
print("="*120)
print("=== Final Answer ===")
print(result.results['report'].result)

<thinking> To plan a 5-day trip from Boston (BOS) to Seattle (SEA) within a $2000 budget, I need to find available flights, check the weather forecast for Seattle during the trip, and consider accommodations, transportation, and activities within the budget. However, I don't have direct access to accommodations, transportation, or activity information. I will start by finding flights that fit the budget.</thinking>


Tool #1: search_flights
<thinking> I have found three flights from Boston (BOS) to Seattle (SEA) within the budget: $299, $349, and $399. Next, I should check the weather forecast for Seattle to ensure the trip is well-planned. This information will help the user decide on the timing of the trip and what to pack.</thinking> 
Tool #2: check_weather
<thinking> The weather forecast for Seattle is sunny with a temperature of 75°F, which is pleasant for a trip. However, the remaining details such as accommodation, local transportation, and activities are not covered by the avai

## Observability: Inspect Graph Execution

In [38]:
# Detailed execution trace
print("=== Graph Execution Details ===")
print(f"Total nodes executed: {len(result.execution_order)}")
print(f"\nExecution path:")

for i, node in enumerate(result.execution_order, 1):
    print(f"\n{i}. Node: {node.node_id}")
    print(f"   Response length: {len(str(node.result.result))} chars")
    print(f"   Preview: {str(node.result.result)[:150]}...")

=== Graph Execution Details ===
Total nodes executed: 2

Execution path:

1. Node: research
   Response length: 858 chars
   Preview: <thinking> The weather forecast for Seattle is sunny with a temperature of 75°F, which is pleasant for a trip. However, the remaining details such as ...

2. Node: report
   Response length: 3248 chars
   Preview: **Final Travel Report: 5-Day Trip from BOS to SEA with $2000 Budget**

---

**Overview:**
This report details a planned 5-day trip from Boston (BOS) t...


## Compare Patterns

In [ ]:
import time

query = "Find flights from NYC to LAX"

patterns = [
    ("Sequential", sequential_graph),
    ("Star", star_graph),
    ("DAG", dag_graph)
]

patterns_metrics = {
    "Sequential": {},
    "Star": {},
    "DAG": {}
}

for name, graph in patterns:
    start = time.time()
    result = graph(query)
    duration = time.time() - start

    metrics = {
        "Execution time": f"{duration:.2f}s",
        "Nodes executed": {result.completed_nodes},
        "Path": {' → '.join([n.node_id for n in result.execution_order])}
    }

    patterns_metrics[name] = metrics


print("/n/n")
print("="*120)
print("=== Pattern Comparison ===")
print(patterns_metrics)

To find flights from NYC (New York City) to LAX (Los Angeles International Airport), follow these steps:

1. Open a web browser on your computer or mobile device.
2. Go to a reputable flight search website, such as Google Flights, Expedia, Kayak, or Skyscanner.
3. In the departure city field, enter "NYC" or select New York City from the dropdown list.
4. In the destination city field, enter "LAX" or select Los Angeles International Airport from the dropdown list.
5. Select your desired travel dates for the departure and return (if applicable).
6. Click on the search button to view the available flight options.
7. Review the search results and compare flights based on factors such as departure and arrival times, duration, price, and airline preferences.
8. Choose the flight that best suits your needs and click on it for more details.
9. If you wish to proceed with booking, follow the instructions provided on the website to complete the reservation.

Remember to consider factors like lay

In [46]:
from pprint import pprint
print("/n/n")
print("="*120)
print("=== Pattern Comparison ===")
pprint(patterns_metrics)

/n/n
=== Pattern Comparison ===
{'DAG': {'Execution time': '2.30s',
         'Nodes executed': {2},
         'Path': {'research → report'}},
 'Sequential': {'Execution time': '3.27s',
                'Nodes executed': {3},
                'Path': {'planner → executor → synthesizer'}},
 'Star': {'Execution time': '41.00s',
          'Nodes executed': {3},
          'Path': {'coordinator → flights → weather → coordinator → flights → '
                   'weather → coordinator → flights → weather → coordinator'}}}
